# Exercise C: Text Classification with RNNs
#### Tzanis Nikolaos mtn2217

In [3]:
# -*- coding: utf-8 -*-
"""

A RNN classifier applied to AG_NEWS dataset

Download dataset:
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset

"""

import torch

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data.dataset import random_split
from torch import nn
from torch.nn import functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from time import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# HYPER-PARAMETERS
MAX_WORDS = 25
EPOCHS = 15
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

In [5]:
######################################################################
# Read dataset files 
# ------------------

train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

In [6]:
######################################################################
# Data processing 
# -----------------------------


tokenizer = get_tokenizer("basic_english")

In [7]:
# All texts are truncated and padded to MAX_WORDS tokens
def collate_batch(batch):
    Y, X = list(zip(*batch))
    Y = torch.tensor(Y) - 1 # Target names in range [0,1,2,3] instead of [1,2,3,4]
    X = [vocab(tokenizer(text)) for text in X]
    # Bringing all samples to MAX_WORDS length. Shorter texts are padded with <PAD> sequences, longer texts are truncated.
    X = [tokens+([vocab['<PAD>']]* (MAX_WORDS-len(tokens))) if len(tokens)<MAX_WORDS else tokens[:MAX_WORDS] for tokens in X]
    return torch.tensor(X, dtype=torch.int32).to(device), Y.to(device) 

In [8]:
train_dataset = [(label,train_data['Title'][i] + ' ' + train_data['Description'][i]) for i,label in enumerate(train_data['Class Index'])]
test_dataset = [(label,test_data['Title'][i] + ' ' + test_data['Description'][i]) for i,label in enumerate(test_data['Class Index'])]

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)

target_classes = ["World", "Sports", "Business", "Sci/Tech"]

In [9]:
def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield tokenizer(text)

In [10]:
# Vocabulary includes all tokens with at least 10 occurrences in the texts
# Special tokens <PAD> and <UNK> are used for padding sequences and unknown words respectively
vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=10, specials=["<PAD>","<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

In [11]:
######################################################################
# Define the model
# ----------------


class model(nn.Module):
    def __init__(self,input_dim, embedding_dim, hidden_dim, output_dim):
        super(model, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=input_dim, embedding_dim=embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        logits = self.linear(output[:,-1])  # The last output of RNN is used for sequence classification
        probs = F.softmax(logits, dim=1)
        return probs

In [12]:
######################################################################
# Initiate an instance of the model
# ---------------------------------


classifier = model(len(vocab), EMBEDDING_DIM, HIDDEN_DIM, len(target_classes)).to(device)
# Define loss function and opimization algorithm
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([param for param in classifier.parameters() if param.requires_grad == True],lr=LEARNING_RATE)

# Count model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('\nModel:')
print(classifier)
print('Total parameters: ',count_parameters(classifier))
print('\n\n')


Model:
model(
  (embedding_layer): Embedding(21254, 100)
  (rnn): RNN(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
)
Total parameters:  2136284





In [13]:
######################################################################
# Define functions to train and evaluate the model
# ------------------------------------------------


def EvaluateModel(model, loss_fn, val_loader):
    model.eval()
    with torch.no_grad():
        Y_actual, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_actual.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_actual = torch.cat(Y_actual)
        Y_preds = torch.cat(Y_preds)
    
    # Returns mean loss, actual labels, predicted labels 
    return torch.tensor(losses).mean(), Y_actual.detach().cpu().numpy(), Y_preds.detach().cpu().numpy()

In [14]:
def TrainModel(model, loss_fn, optimizer, train_loader, epochs):
    total_time = 0
    for i in range(1, epochs+1):
        start = time()
        model.train()
        print('Epoch:',i)
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        end = time()
        total_time += end - start

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
    
    return total_time/epochs

## Question 1

The current implementation of the model is based on a one-directional RNN model with 1 layer. In order to test different models we have to change the class that is used to define the model.

Firstly, the option to choose between RNN or LSTM is added. An 'if' statement checks the param 'neural_network' when a new model is defined. Depending on the option, an appropriate neural network is initialized.

Secondly, we have to add an option for one-directional or bidirectional. This is once again given as a param and when creating the rnn or lstm model is used to create an appropriate neural network. Note that if the nn is bidirectional then the hidden dimension has to be doubled. This is specified in the nn.Linear function call.

Lastly, the number of layer is also given as a param which is being used at the nn.RNN or nn.LSTM for RNNs or LSTMs respectively.

In [15]:
class model(nn.Module):
    def __init__(self,input_dim, embedding_dim, hidden_dim, output_dim, bidirectional=False, neural_network='RNN', num_layers=1):
        super(model, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=input_dim, embedding_dim=embedding_dim)
        if neural_network=='RNN':
            self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
            self.linear = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        elif neural_network=='LSTM':
            self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
            self.linear = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        else:
            print('Not supported NN')
            return 
        
    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        logits = self.linear(output[:,-1])  # The last output of RNN is used for sequence classification
        probs = F.softmax(logits, dim=1)
        return probs

In order to evaluate the performance of each model, we create a function based on the given code. This function trains each model and then evaluates it while returning the accuracy, total parameters, time cost, the classification report, the misclassified texts and the predictions for the model in question.

In [16]:
def TrainAndEvaluateModelAndReturnMetrics(bidirectional, neural_network, num_layers):

    classifier = model(len(vocab), EMBEDDING_DIM, HIDDEN_DIM, len(target_classes), bidirectional=bidirectional, neural_network=neural_network, num_layers=num_layers).to(device)
    # Define loss function and opimization algorithm
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam([param for param in classifier.parameters() if param.requires_grad == True],lr=LEARNING_RATE)
    total_parameters = count_parameters(classifier)

    time_cost = TrainModel(classifier, loss_fn, optimizer, train_loader, EPOCHS)

    _, Y_actual, Y_preds = EvaluateModel(classifier, loss_fn, test_loader)

    accuracy = accuracy_score(Y_actual, Y_preds)

    report = classification_report(Y_actual, Y_preds, target_names=target_classes)

    cmatrix = confusion_matrix(Y_actual, Y_preds)

    misclassified = np.where(Y_actual != Y_preds)[0]

    return [np.round(accuracy, decimals=2), total_parameters, np.round(time_cost, decimals=2)], report, cmatrix, misclassified, Y_actual, Y_preds


Next we call the above function for each model and get the stats for each one of them.

In [17]:
rnn1 = TrainAndEvaluateModelAndReturnMetrics(bidirectional=False, neural_network='RNN', num_layers=1)
rnn1Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=1)
rnn2Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=2)
lstm1 = TrainAndEvaluateModelAndReturnMetrics(bidirectional=False, neural_network='LSTM', num_layers=1)
lstm1Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=1)
lstm2Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=2)

Epoch: 1


100%|██████████| 118/118 [00:09<00:00, 12.47it/s]


Train Loss : 1.297
Epoch: 2


100%|██████████| 118/118 [00:09<00:00, 13.11it/s]


Train Loss : 1.047
Epoch: 3


...

Train Loss : 0.816
Epoch: 14


100%|██████████| 118/118 [00:30<00:00,  3.85it/s]


Train Loss : 0.814
Epoch: 15


100%|██████████| 118/118 [00:30<00:00,  3.85it/s]


Train Loss : 0.811


The last part is to create a pandas dataframe in order to display the results.

In [18]:
results_df = pd.DataFrame({'1RNN': rnn1[0], '1Bi-RNN': rnn1Bi[0], '2Bi-RNN': rnn2Bi[0], '1LSTM': lstm1[0], '1Bi-LSTM': lstm1Bi[0], '2Bi-LSTM': lstm2Bi[0]})
results_df = results_df.rename(index={0:'Accuracy', 1:'Parameters', 2:'Time Cost'})
display(results_df)

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,0.87,0.86,0.86,0.88,0.88,0.89
Parameters,2136284.00,2147164.00,2171996.00,2168156.00,2210908.00,2310236.00
Time Cost,9.12,14.37,22.07,11.04,17.70,30.61


From the results it is obvious that as we increase the complexity of a model, the same happens with its accuracy. Models with more parameters tend to have better results. Of course, LSTM models have better accuracy than RNN models accross the board, regardless of the number of parameters.

We can also see that models with multiple layers tend to use more resources and need more time to train. The same goes for models with more parameters as we can see they also take more time to train.

Finally, we can see that using 2 layers instead of 2 improves the accuracy of both the RNN and LSTM models.

## Question 2

Using the output of the function that we called earlier and especially the misclassified texts for each model we create a variable that stores all common misclassifications between all the models and then display the length of that list (which practically shows how many texts were misclassified by all the models).

In [19]:
common_misclassified = list(set(rnn1[3]).intersection(rnn1Bi[3], rnn2Bi[3], lstm1[3], lstm1Bi[3], lstm2Bi[3]))
len(common_misclassified)

326

Then, we pick a random number from the list of misclassified texts and diplay the text.

In [20]:
import random

random_text = common_misclassified[random.randint(0, len(common_misclassified))]
print(f"A sample text that was misclassified by every model was:\n{test_data['Title'][random_text]} {test_data['Description'][random_text]}")

A sample text that was misclassified by every model was:
Telescope snaps distant 'planet' The first direct image of a planet circling another star may have been obtained by a US-European team of astronomers.


Next we go through the misclassifications to find out what their original category was in order to specify the misclassified texts per category.

In [21]:
misclassifications_per_category = {
    'World': 0,
    'Sports:': 0,
    'Business:': 0,
    'Sci/Tech:': 0
}
for index in common_misclassified:
    if test_data['Class Index'][index] == 1:
        misclassifications_per_category['World'] += 1
    elif test_data['Class Index'][index] == 2:
        misclassifications_per_category['Sports:'] += 1
    elif test_data['Class Index'][index] == 3:
        misclassifications_per_category['Business:'] += 1
    elif test_data['Class Index'][index] == 4:
        misclassifications_per_category['Sci/Tech:'] += 1
    else:
        pass

print(f'Misclassifications per category:\n{misclassifications_per_category}')

Misclassifications per category:
{'World': 99, 'Sports:': 13, 'Business:': 127, 'Sci/Tech:': 87}


Then we look up all predictions to find the numbers of times that each pair of correct category - wrong prediction showed up.

In [22]:
models  = [rnn1, rnn1Bi, rnn2Bi, lstm1, lstm1Bi, lstm2Bi]
mistake_pairs = {}

for specific_model in models:
    for index in common_misclassified:
        try:
            mistake_pairs[(specific_model[4][index]+1, specific_model[5][index]+1)] += 1
        except:
            mistake_pairs[(specific_model[4][index]+1, specific_model[5][index]+1)] = 1

And then we prin that result.

In [23]:
most_common_pair = max(mistake_pairs, key=lambda k: mistake_pairs[k])
new_pair = []
if most_common_pair[0] == 1:
    new_pair.append('World')
elif  most_common_pair[0] == 2:
    new_pair.append('Sports')
elif most_common_pair[0] == 3:
    new_pair.append('Business')
elif  most_common_pair[0] == 4:
    new_pair.append('Sci/Tech')
else:
    pass
if most_common_pair[1] == 1:
    new_pair.append('World')
elif  most_common_pair[1] == 2:
    new_pair.append('Sports')
elif most_common_pair[1] == 3:
    new_pair.append('Business')
elif   most_common_pair[1] == 4:
    new_pair.append('Sci/Tech')
else:
    pass
print(f'The most common pair of correct category - wrong prediction was: {(new_pair[0], new_pair[1])}, which appeared a total of {mistake_pairs[most_common_pair]} times.' )

The most common pair of correct category - wrong prediction was: ('Business', 'Sci/Tech'), which appeared a total of 555 times.


## Question 3

For the next question the number of MAX_WORDS is set to 50 and then we call again the function to train and evaluate each model.

In [24]:
MAX_WORDS = 50

rnn1 = TrainAndEvaluateModelAndReturnMetrics(bidirectional=False, neural_network='RNN', num_layers=1)
rnn1Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=1)
rnn2Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=2)
lstm1 = TrainAndEvaluateModelAndReturnMetrics(bidirectional=False, neural_network='LSTM', num_layers=1)
lstm1Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=1)
lstm2Bi = TrainAndEvaluateModelAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=2)

Epoch: 1


100%|██████████| 118/118 [00:14<00:00,  8.08it/s]


Train Loss : 1.380
Epoch: 2


100%|██████████| 118/118 [00:14<00:00,  8.42it/s]


Train Loss : 1.362
Epoch: 3


...

Train Loss : 0.823
Epoch: 14


100%|██████████| 118/118 [01:04<00:00,  1.83it/s]


Train Loss : 0.822
Epoch: 15


100%|██████████| 118/118 [01:04<00:00,  1.83it/s]


Train Loss : 0.822


Then we display the results in the same form as the question 1.

In [25]:
results_df = pd.DataFrame({'1RNN': rnn1[0], '1Bi-RNN': rnn1Bi[0], '2Bi-RNN': rnn2Bi[0], '1LSTM': lstm1[0], '1Bi-LSTM': lstm1Bi[0], '2Bi-LSTM': lstm2Bi[0]})
results_df = results_df.rename(index={0:'Accuracy', 1:'Parameters', 2:'Time Cost'})
display(results_df)

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,0.49,0.35,0.41,0.89,0.88,0.89
Parameters,2136284.00,2147164.00,2171996.00,2168156.00,2210908.00,2310236.00
Time Cost,14.08,24.16,40.57,19.99,35.26,64.48


As we can see the accuracy of the RNN models was reduced significantly and this is probably attributed to the vanishing gradient problem. On the contrary, the LSTM models are not affected by this issue as they can 'remember' important information from each timestep. The number of parameters is not affected as the structure of the RNN or LSTM is not changed based on the MAX_WORDS value.

## Question 4

For this question we have to modify again the definition of the model. This time we have to add a param that specifies if pretrained embeddings are going to be used. Note that these embeddings could be modified while the model is being trained (freeze=false).

In [26]:
class model(nn.Module):
    def __init__(self,input_dim, embedding_dim, hidden_dim, output_dim, bidirectional=False, neural_network='RNN', num_layers=1, pretrained_embeddings = None, freeze = False):
        super(model, self).__init__()

        if pretrained_embeddings == None:
            self.embedding_layer = nn.Embedding(num_embeddings=input_dim, embedding_dim=embedding_dim)
        else:
            self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=freeze)
        if neural_network=='RNN':
            self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
            self.linear = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        elif neural_network=='LSTM':
            self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
            self.linear = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        else:
            print('Not supported NN')
            return 
        
    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        logits = self.linear(output[:,-1])  # The last output of RNN is used for sequence classification
        probs = F.softmax(logits, dim=1)
        return probs

Then we import the glove embeddings and set the MAX_WORDS value back to 25.

In [27]:
from torchtext.vocab import GloVe

glove = GloVe(name='6B', dim=100)

MAX_WORDS = 25

And modify the function that trains and evaluates the models to take into account if we used pretrained embeddings and if the have to be frozen.

In [28]:
def TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional, neural_network, num_layers, pretrained_embeddings, freeze):

    classifier = model(len(vocab), EMBEDDING_DIM, HIDDEN_DIM, len(target_classes), bidirectional=bidirectional, neural_network=neural_network, num_layers=num_layers, pretrained_embeddings=pretrained_embeddings, freeze=freeze).to(device)
    # Define loss function and opimization algorithm
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam([param for param in classifier.parameters() if param.requires_grad == True],lr=LEARNING_RATE)
    total_parameters = count_parameters(classifier)

    time_cost = TrainModel(classifier, loss_fn, optimizer, train_loader, EPOCHS)

    _, Y_actual, Y_preds = EvaluateModel(classifier, loss_fn, test_loader)

    accuracy = accuracy_score(Y_actual, Y_preds)

    report = classification_report(Y_actual, Y_preds, target_names=target_classes)

    cmatrix = confusion_matrix(Y_actual, Y_preds)

    return [np.round(accuracy, decimals=2), total_parameters, np.round(time_cost, decimals=2)], report, cmatrix

After this is done, we train and evaluate each model once again.

In [29]:
rnn1 = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=False, neural_network='RNN', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=False)
rnn1Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=False)
rnn2Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=2, pretrained_embeddings=glove.vectors.clone().detach(), freeze=False)
lstm1 = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=False, neural_network='LSTM', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=False)
lstm1Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=False)
lstm2Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=2, pretrained_embeddings=glove.vectors.clone().detach(), freeze=False)

Epoch: 1


100%|██████████| 118/118 [00:26<00:00,  4.45it/s]


Train Loss : 1.248
Epoch: 2


100%|██████████| 118/118 [00:26<00:00,  4.47it/s]


Train Loss : 0.952
Epoch: 3


...

Train Loss : 0.806
Epoch: 14


100%|██████████| 118/118 [00:51<00:00,  2.29it/s]


Train Loss : 0.804
Epoch: 15


100%|██████████| 118/118 [00:51<00:00,  2.30it/s]


Train Loss : 0.802


Then we display the results for each model using a pandas df.

In [30]:
results_df = pd.DataFrame({'1RNN': rnn1[0], '1Bi-RNN': rnn1Bi[0], '2Bi-RNN': rnn2Bi[0], '1LSTM': lstm1[0], '1Bi-LSTM': lstm1Bi[0], '2Bi-LSTM': lstm2Bi[0]})
results_df = results_df.rename(index={0:'Accuracy', 1:'Parameters', 2:'Time Cost'})
display(results_df)

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,0.88,0.89,0.88,0.89,0.89,0.89
Parameters,40010884.00,40021764.00,40046596.00,40042756.00,40085508.00,40184836.00
Time Cost,26.36,30.96,38.61,29.06,36.66,51.48


As we can see the accuracy of all the models increased significantly when we used pretrained embeddings. The number of parameters also increased as the embedding layer is alot larger because of the addition of the Glove embeddings.

## Question 5

For this question we will freeze the pretrained embeddings so they can't be changed during the training process.

In [31]:
rnn1 = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=False, neural_network='RNN', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=True)
rnn1Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=True)
rnn2Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=2, pretrained_embeddings=glove.vectors.clone().detach(), freeze=True)
lstm1 = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=False, neural_network='LSTM', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=True)
lstm1Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=1, pretrained_embeddings=glove.vectors.clone().detach(), freeze=True)
lstm2Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=2, pretrained_embeddings=glove.vectors.clone().detach(), freeze=True)

Epoch: 1


100%|██████████| 118/118 [00:07<00:00, 14.79it/s]


Train Loss : 1.349
Epoch: 2


100%|██████████| 118/118 [00:07<00:00, 15.27it/s]


Train Loss : 1.277
Epoch: 3


...

Train Loss : 0.932
Epoch: 14


100%|██████████| 118/118 [00:33<00:00,  3.52it/s]


Train Loss : 0.924
Epoch: 15


100%|██████████| 118/118 [00:33<00:00,  3.53it/s]


Train Loss : 0.919


In [32]:
results_df = pd.DataFrame({'1RNN': rnn1[0], '1Bi-RNN': rnn1Bi[0], '2Bi-RNN': rnn2Bi[0], '1LSTM': lstm1[0], '1Bi-LSTM': lstm1Bi[0], '2Bi-LSTM': lstm2Bi[0]})
results_df = results_df.rename(index={0:'Accuracy', 1:'Parameters', 2:'Time Cost'})
display(results_df)

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,0.59,0.64,0.66,0.79,0.78,0.80
Parameters,10884.00,21764.00,46596.00,42756.00,85508.00,184836.00
Time Cost,7.67,11.90,19.82,10.71,18.43,33.31


This time the accuracy of the models was reduced but it's important to note that the number of parameters was also a lot smaller for each model.

## Question 6

In order to use the IMDB Dataset first of all we have to read it from the folder where it is stored. Then, by using sklearn's function train_test_split we split that dataframe to train and test with a 80:20 ratio. The random_state is used to always split the data in the same way.

In [33]:
from sklearn.model_selection import train_test_split

imdb_data = pd.read_csv('../data/IMDB Dataset.csv')
train_data, test_data = train_test_split(imdb_data, test_size=0.2, random_state=42)

The function in the next cell will help create the dataset from the raw data. 1 will be the label for 'negative' and 2 the label for 'positive' reviews. The new dataset will be comprised of tuples of the form (1, text), where the first number is the sentiment and the text refers to the text of the review. The data has to be formed in that way so the model can use them.

In [34]:
def createDataset(data):
    dataset = []
    for i in range(len(data['review'])):
        if data['sentiment'].iloc[i] == 'negative':
            dataset.append((1, data['review'].iloc[i]))
        else:
            dataset.append((2, data['review'].iloc[i]))
    return dataset

The function we created earlier is called for both the training and the test set.

In [35]:
train_dataset = createDataset(train_data)
test_dataset = createDataset(test_data)

Then the vocabulary is created (this function is not changed).

In [36]:
vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, test_dataset]), min_freq=10, specials=["<PAD>","<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

Then we call the Dataloader function for the train and test set (unchanged also) and specify the target classes which in this case will be 'negative' and 'positive'.

In [37]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)

target_classes = ["negative", "positive"]

The last step is to train the models.

In [38]:
rnn1 = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=False, neural_network='RNN', num_layers=1, pretrained_embeddings=None, freeze=False)
rnn1Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=1, pretrained_embeddings=None, freeze=False)
rnn2Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='RNN', num_layers=2, pretrained_embeddings=None, freeze=False)
lstm1 = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=False, neural_network='LSTM', num_layers=1, pretrained_embeddings=None, freeze=False)
lstm1Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=1, pretrained_embeddings=None, freeze=False)
lstm2Bi = TrainAndEvaluateModelWithPretrainedEmbeddingsAndReturnMetrics(bidirectional=True, neural_network='LSTM', num_layers=2, pretrained_embeddings=None, freeze=False)

Epoch: 1


100%|██████████| 40/40 [00:07<00:00,  5.51it/s]


Train Loss : 0.696
Epoch: 2


100%|██████████| 40/40 [00:06<00:00,  5.81it/s]


Train Loss : 0.688
Epoch: 3


...

Train Loss : 0.450
Epoch: 14


100%|██████████| 40/40 [00:13<00:00,  2.87it/s]


Train Loss : 0.434
Epoch: 15


100%|██████████| 40/40 [00:13<00:00,  2.88it/s]


Train Loss : 0.420


In [39]:
results_df = pd.DataFrame({'1RNN': rnn1[0], '1Bi-RNN': rnn1Bi[0], '2Bi-RNN': rnn2Bi[0], '1LSTM': lstm1[0], '1Bi-LSTM': lstm1Bi[0], '2Bi-LSTM': lstm2Bi[0]})
results_df = results_df.rename(index={0:'Accuracy', 1:'Parameters', 2:'Time Cost'})
display(results_df)

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,0.71,0.7,0.70,0.72,0.72,0.72
Parameters,2917254.00,2928006.0,2952838.00,2949126.00,2991750.00,3091078.00
Time Cost,6.79,8.3,10.79,7.34,9.55,13.89
